# Import Modules

In [47]:
import os
import sys
import pandas as pd
import numpy as np
import datetime as dt
import itertools as it
import csv
import ast
from pprint import pprint
from prettytable import PrettyTable
import matplotlib
import matplotlib.pylab as plt
import matplotlib.colors as mcolors
import seaborn as sns

pdir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # for jupyter-notebook
sys.path.insert(1, pdir)
from myutils import *

%config InlineBackend.figure_format = 'retina'  # 提高 jupyter notebook 的圖形顯示解析度
plt.style.use('ggplot')

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

# Define Basic Functions

In [48]:
def set_data(df, mode='pcap', tz=0):
    if mode == 'pcap':
        df['seq'] = df['seq'].astype('Int32')
        # df['rpkg'] = df['rpkg'].astype('Int8')
        df['frame_id'] = df['frame_id'].astype('Int32')
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df['xmit_time'] = pd.to_datetime(df['xmit_time'])
        df['arr_time'] = pd.to_datetime(df['arr_time'])
        df['Timestamp_epoch'] = df['Timestamp_epoch'].astype('float32')
        df['xmit_time_epoch'] = df['xmit_time_epoch'].astype('float32')
        df['arr_time_epoch'] = df['arr_time_epoch'].astype('float32')
        df['lost'] = df['lost'].astype('boolean')
        df['excl'] = df['excl'].astype('boolean')
        df['latency'] = df['latency'].astype('float32')
    # LTE (E-UTRA) & NR
    def nr_serv_cel(row):
        pos = row.serv_cel_pos
        if pos == 255:
            return 65535, -160, -50
        else:
            return row[f'PCI{pos}'], row[f'RSRP{pos}'], row[f'RSRQ{pos}']
    if mode == 'lte':
        df = df.rename(columns={
            'RSRP(dBm)': 'RSRP',
            'RSRQ(dB)': 'RSRQ',
            'Serving Cell Index': 'serv_cel_index',
            'Number of Neighbor Cells': 'num_neigh_cels',
            'Number of Detected Cells': 'num_cels',
            }).reindex(['Timestamp','type_id','PCI','RSRP','RSRQ','serv_cel_index','EARFCN','NR_ARFCN',
                        'num_cels','num_neigh_cels','serv_cel_pos','PCI0','RSRP0','RSRQ0',
                        *df.columns.to_list()[df.columns.get_loc('PCI1'):]], axis=1)
        df.loc[df['serv_cel_index'] == '(MI)Unknown', 'serv_cel_index'] = '3_SCell'
        df['num_cels'] = df['num_neigh_cels'] + 1
        ## set dtypes
        df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
        df['type_id'] = df['type_id'].astype('category')
        df['serv_cel_index'] = df['serv_cel_index'].astype('category')
        df['EARFCN'] = df['EARFCN'].astype('Int32')
        df['NR_ARFCN'] = df['NR_ARFCN'].astype('Int32')
        df['num_cels'] = df['num_cels'].astype('UInt8')
        df['num_neigh_cels'] = df['num_neigh_cels'].astype('UInt8')
        df['serv_cel_pos'] = df['serv_cel_pos'].astype('UInt8')
        for tag in df.columns:
            if tag.startswith('PCI'):
                df[tag] = df[tag].astype('Int32')
            if tag.startswith(('RSRP','RSRQ')):
                df[tag] = df[tag].astype('float32')
    if mode == 'nr':
        if df.empty:
            df = pd.DataFrame(columns=['Timestamp','type_id','PCI','RSRP','RSRQ','serv_cel_index','EARFCN','NR_ARFCN',
                                       'num_cels','num_neigh_cels','serv_cel_pos','PCI0','RSRP0','RSRQ0'])
        else:
            df = df.rename(columns={
                'Raster ARFCN': 'NR_ARFCN',
                'Serving Cell Index': 'serv_cel_pos',
                'Num Cells': 'num_cels',
                }).reindex(['Timestamp','type_id','PCI','RSRP','RSRQ','serv_cel_index','EARFCN','NR_ARFCN',
                            'num_cels','num_neigh_cels','serv_cel_pos','PCI0','RSRP0','RSRQ0',
                            *df.columns.to_list()[df.columns.get_loc('PCI1'):]], axis=1)
            df.loc[df['serv_cel_pos'] != 255, 'serv_cel_index'] = 'PSCell'
            df[['PCI','RSRP','RSRQ']] = df.apply(nr_serv_cel, axis=1, result_type='expand')
            df.loc[df['serv_cel_pos'] == 255, 'num_neigh_cels'] = df['num_cels']
            df.loc[df['serv_cel_pos'] != 255, 'num_neigh_cels'] = df['num_cels'] - 1
        ## set dtypes
        df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
        df['type_id'] = df['type_id'].astype('category')
        df['serv_cel_index'] = df['serv_cel_index'].astype('category')
        df['EARFCN'] = df['EARFCN'].astype('Int32')
        df['NR_ARFCN'] = df['NR_ARFCN'].astype('Int32')
        df['num_cels'] = df['num_cels'].astype('UInt8')
        df['num_neigh_cels'] = df['num_neigh_cels'].astype('UInt8')
        df['serv_cel_pos'] = df['serv_cel_pos'].astype('UInt8')
        for tag in df.columns:
            if tag.startswith('PCI'):
                df[tag] = df[tag].astype('Int32')
            if tag.startswith(('RSRP','RSRQ')):
                df[tag] = df[tag].astype('float32')
    return df

In [49]:
# # root = '/home/wmnlab/D/database/'
# root = '/Users/jackbedford/Desktop/MOXA/Code/data/'
# paths = [s for s in os.listdir(root) if s.startswith('202') and os.path.isdir(os.path.join(root, s))]

# md_files = []

# for date_dir in paths:
#     datedir = os.path.join(root, date_dir)
#     md_file = [os.path.join(datedir, s) for s in os.listdir(datedir) if s.endswith('.md')]
#     md_files += md_file

# md_files = sorted(md_files)

# # Select dates
# # md_files = md_files[:-1]
# # md_files = [md_files[2]]

# pprint(md_files)

# Set Dates

In [50]:
dates = ['2023-06-12',
         '2023-06-13',
         '2023-06-15',]

# root = '/home/wmnlab/D/database/'
root = '/Users/jackbedford/Desktop/MOXA/Code/data/'
paths = [s for s in dates if os.path.isdir(os.path.join(root, s))]

md_files = []

for date_dir in paths:
    datedir = os.path.join(root, date_dir)
    md_file = [os.path.join(datedir, s) for s in os.listdir(datedir) if s.endswith('.md')]
    md_files += md_file

md_files = sorted(md_files)

pprint(md_files)

['/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/2023-06-12.md',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/2023-06-13.md',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/2023-06-15.md']


In [51]:
class Experiment():
    
    def __init__(self, dir_name, setting):
        
        self.dir_name = dir_name
        self.setting = setting
        
    def __repr__(self):
        return f'{self.dir_name} -> {self.setting}'

exp_list = []

for md_file in md_files:
    
    md_f = open(md_file, 'r')
    lines = []
    l = md_f.readline()[:-1]
    
    while l:
        
        if l == '#endif':
            break
        lines.append(l)
        l = md_f.readline()[:-1]
        
    md_f.close()
    
    for exp, dic in zip(lines[::2], lines[1::2]):
        
        dictionary = ast.literal_eval(dic)
        parent_dir = os.path.dirname(md_file)
        experiment = Experiment(dir_name=os.path.join(parent_dir, exp), setting=dictionary)
        exp_list.append(experiment)
        
pprint(exp_list)

[/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone -> {'sm00': 'B3B7B8', 'sm01': 'B3', 'sm02': 'B7', 'sm03': 'B8', 'sm04': 'B3B7', 'sm05': 'B3B8', 'sm06': 'B7B8', 'sm07': 'LTE'},
 /Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/Bandlock_8_Schemes_Phone -> {'sm00': 'B3B7B8', 'sm01': 'B3', 'sm02': 'B7', 'sm03': 'B8', 'sm04': 'B3B7', 'sm05': 'B3B8', 'sm06': 'B7B8', 'sm07': 'LTE'},
 /Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_B1_B3_B7_B8_Dupl_Phone -> {'sm00': 'B1', 'sm01': 'B1', 'sm02': 'B3', 'sm03': 'B3', 'sm04': 'B7', 'sm05': 'B7', 'sm06': 'B8', 'sm07': 'B8'},
 /Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_All_B3_B7_B8_Dupl_Phone -> {'sm00': 'All', 'sm01': 'All', 'sm02': 'B3', 'sm03': 'B3', 'sm04': 'B7', 'sm05': 'B7', 'sm06': 'B8', 'sm07': 'B8'},
 /Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTE_B3_B7_B8_Dupl_Phone -> {'sm00': 'LTE', 'sm01': 'LTE', 'sm02': 'B3', 'sm03': 'B3', 'sm04': 'B7', 'sm05': 'B7', 

## Processing

In [53]:
class ExpTable():
    
    def __init__(self, exp_name, table):
        
        self.exp_name = exp_name
        self.table = table
    
    def __repr__(self):
        return self.exp_name

table_list = []

for exp in exp_list:
    exp_dir = exp.dir_name
    exp_setting = exp.setting
    
    table = pd.DataFrame(columns=exp_setting.keys())
    table.loc['Scheme'] = exp_setting.values()
    
    for dev, schm in exp_setting.items():
        dev_dir = os.path.join(exp_dir, dev)
        trace_list = [s for s in sorted(os.listdir(dev_dir)) if os.path.isdir(os.path.join(dev_dir, s))]
        
        for trace in trace_list:
            data_dir = os.path.join(dev_dir, trace, 'data')
            print(data_dir)
            
            data = os.path.join(data_dir, [s for s in os.listdir(data_dir) if s.startswith('diag_log') and s.endswith('_nr_ml1.pkl')][0])
            # print(data)
            df_nr = pd.read_pickle(data)
            df_nr = set_data(df_nr, mode='nr', tz=8)
            
            data = os.path.join(data_dir, [s for s in os.listdir(data_dir) if s.startswith('diag_log') and s.endswith('_rrc.pkl')][0])
            # print(data)
            df_ho = pd.read_pickle(data)
            df_ho, _, _ = mi_parse_ho(df_ho, tz=8)
            df_ho['ho_type0'] = df_ho['ho_type'].astype('string')
            df_ho.loc[df_ho['cause'].notna(), 'ho_type0'] = df_ho['ho_type'].astype('string') + '_' + df_ho['cause'].astype('string')
            df_ho['ho_type0'] = df_ho['ho_type0'].astype('category')
            df_ho.to_csv(os.path.join(data_dir, 'diag_handover_info.csv'), index=False)
            
            check_signal = 'O'
            if sum(df_ho['cause'] == 'rlc-MaxNumRetx (2)') > 0:
                check_signal = '∆'
            if df_nr.empty:
                check_signal = 'X'
            table.loc[trace, dev] = check_signal
            
    exp_table = ExpTable(exp_name=os.path.join(os.path.basename(os.path.dirname(exp_dir)), os.path.basename(exp_dir)), table=table)
    table_list.append(exp_table)

/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm00/#01/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm00/#02/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm00/#03/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm00/#04/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm00/#05/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm00/#06/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm00/#07/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm00/#08/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm00/#09/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm00/#10/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm

/var/folders/vy/w6g3hxgj76s0gzty27t2d6gm0000gn/T/ipykernel_59704/4044466450.py:63: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[df['serv_cel_pos'] != 255, 'num_neigh_cels'] = df['num_cels'] - 1


/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm07/#09/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-12/Bandlock_8_Schemes_Phone/sm07/#10/data


/var/folders/vy/w6g3hxgj76s0gzty27t2d6gm0000gn/T/ipykernel_59704/4044466450.py:63: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[df['serv_cel_pos'] != 255, 'num_neigh_cels'] = df['num_cels'] - 1


/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/Bandlock_8_Schemes_Phone/sm00/#02/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/Bandlock_8_Schemes_Phone/sm00/#03/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/Bandlock_8_Schemes_Phone/sm00/#04/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/Bandlock_8_Schemes_Phone/sm00/#05/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/Bandlock_8_Schemes_Phone/sm00/#06/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/Bandlock_8_Schemes_Phone/sm00/#07/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/Bandlock_8_Schemes_Phone/sm00/#08/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/Bandlock_8_Schemes_Phone/sm00/#09/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/Bandlock_8_Schemes_Phone/sm00/#10/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/Bandlock_8_Schemes_Phone/sm01/#02/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-13/Bandlock_8_Schemes_Phone/sm

...

/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTE_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm05/#03/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTE_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm05/#04/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTE_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm06/#01/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTE_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm06/#02/data


/var/folders/vy/w6g3hxgj76s0gzty27t2d6gm0000gn/T/ipykernel_59704/4044466450.py:63: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[df['serv_cel_pos'] != 255, 'num_neigh_cels'] = df['num_cels'] - 1


/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTE_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm06/#03/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTE_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm06/#04/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTE_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm07/#01/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTE_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm07/#02/data


/var/folders/vy/w6g3hxgj76s0gzty27t2d6gm0000gn/T/ipykernel_59704/4044466450.py:63: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[df['serv_cel_pos'] != 255, 'num_neigh_cels'] = df['num_cels'] - 1


/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTE_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm07/#03/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTE_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm07/#04/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTEb1_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm00/#01/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTEb1_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm00/#02/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTEb1_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm00/#03/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTEb1_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm00/#04/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTEb1_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm01/#01/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTEb1_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm01/#02/data
/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_LTEb1_LTEb3_LTEb7_LTEb8_Dupl_Phone/sm01/#03/data
/User

In [46]:
print('O: RF Module is always on.')
print('∆: RF Module was turned off midway. "SCG Failure: rlc-MaxNumRetx (2)" occurs!')
print('X: RF Module has never been turned on.')
print()

for table in table_list:
    print(table.exp_name)
    display(table.table)

O: RF Module is always on.
∆: RF Module was turned off midway. "SCG Failure: rlc-MaxNumRetx (2)" occurs!
X: RF Module has never been turned on.

2023-06-12/Bandlock_8_Schemes_Phone


,sm00,sm01,sm02,sm03,sm04,sm05,sm06,sm07
Scheme,B3B7B8,B3,B7,B8,B3B7,B3B8,B7B8,LTE
#01,O,O,O,O,O,O,O,O
#02,O,O,O,O,O,∆,O,∆
#03,O,O,O,O,O,O,O,O
#04,O,O,∆,O,O,O,O,O
#05,O,O,∆,O,O,O,∆,O
#06,O,O,O,O,O,O,O,O
#07,O,O,O,O,O,O,O,O
#08,O,O,∆,O,O,O,∆,∆
#09,O,O,O,O,O,O,O,∆


2023-06-13/Bandlock_8_Schemes_Phone


,sm00,sm01,sm02,sm03,sm04,sm05,sm06,sm07
Scheme,B3B7B8,B3,B7,B8,B3B7,B3B8,B7B8,LTE
#02,O,O,O,O,O,O,O,O
#03,O,O,O,O,O,O,O,O
#04,O,O,O,O,O,O,O,O
#05,O,O,O,O,O,O,O,O
#06,O,O,O,O,O,O,O,O
#07,O,O,O,O,O,O,O,O
#08,O,O,O,O,O,O,O,O
#09,O,O,O,O,O,O,O,O
#10,O,O,O,O,O,O,O,O


2023-06-15/Bandlock_B1_B3_B7_B8_Dupl_Phone


...

,sm00,sm01,sm02,sm03,sm04,sm05,sm06,sm07
Scheme,LTE,LTE,B3,B3,B7,B7,B8,B8
#01,X,X,O,O,O,O,O,O
#02,X,X,O,O,O,∆,∆,O
#03,X,X,O,O,O,O,O,O
#04,X,X,O,O,O,O,O,O
#05,X,X,O,O,O,O,O,O
#06,X,X,O,O,O,O,O,O


2023-06-15/Bandlock_LTE_LTEb3_LTEb7_LTEb8_Dupl_Phone


,sm00,sm01,sm02,sm03,sm04,sm05,sm06,sm07
Scheme,LTE,LTE,LTEb3,LTEb3,LTEb7,LTEb7,LTEb8,LTEb8
#01,X,X,O,O,O,O,O,O
#02,X,X,X,X,X,X,X,X
#03,X,X,X,X,X,X,X,X
#04,X,X,X,X,X,X,X,X


2023-06-15/Bandlock_LTEb1_LTEb3_LTEb7_LTEb8_Dupl_Phone


,sm00,sm01,sm02,sm03,sm04,sm05,sm06,sm07
Scheme,LTEb1,LTEb1,LTEb3,LTEb3,LTEb7,LTEb7,LTEb8,LTEb8
#01,X,X,X,X,X,X,X,X
#02,X,X,X,X,X,X,X,X
#03,X,X,X,X,X,X,X,X
#04,X,X,X,X,X,X,X,X
